In [33]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

In [34]:
tweets = pd.read_csv('Tweets.csv',sep=',')
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [35]:
data = tweets[['text','airline_sentiment']]
data.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [36]:
data = data[data.airline_sentiment !="neutral"]
data.head()

,text,airline_sentiment
1,@VirginAmerica plus you've added commercials t...,positive
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative
5,@VirginAmerica seriously would pay $30 a fligh...,negative
6,"@VirginAmerica yes, nearly every time I fly VX...",positive


In [37]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]','',x))
data['text'].head()

1    virginamerica plus youve added commercials to ...
3    virginamerica its really aggressive to blast o...
4    virginamerica and its a really big bad thing a...
5    virginamerica seriously would pay 30 a flight ...
6    virginamerica yes nearly every time i fly vx t...
Name: text, dtype: object

In [38]:
data[ data['airline_sentiment'] == 'positive'].size,data[ data['airline_sentiment'] == 'negative'].size


(4726, 18356)

In [39]:
data.shape

(11541, 2)

In [40]:
data['airline_sentiment'].value_counts()

negative    9178
positive    2363
Name: airline_sentiment, dtype: int64

tokenization

In [41]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

neural network

In [42]:
X.shape

(11541, 32)

In [43]:
embed_dim = 128
lstm_out = 196
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Dropout(0.5))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 32, 128)           256000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
data.airline_sentiment.value_counts()

negative    9178
positive    2363
Name: airline_sentiment, dtype: int64

In [45]:
Y = pd.get_dummies(data['airline_sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7732, 32) (7732, 2)
(3809, 32) (3809, 2)


In [46]:
data.airline_sentiment.value_counts()

negative    9178
positive    2363
Name: airline_sentiment, dtype: int64

In [47]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.33,random_state=42)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((7732, 32), (3809, 32), (7732, 2), (3809, 2))

selecting data for validation

In [48]:
X_val = X_train[:500]
Y_val = Y_train[:500]

partial_X_train = X_train[500:]
partial_Y_train = Y_train[500:]

training network

In [ ]:
batch_size = 512
history = model.fit(partial_X_train,partial_Y_train,epochs=10,batch_size=batch_size,validation_data=(X_val,Y_val))

Train on 7232 samples, validate on 500 samples
Epoch 1/10
7232/7232 [==============================] - 41s 6ms/step - loss: 0.5707 - acc: 0.7782 - val_loss: 0.4524 - val_acc: 0.7940
Epoch 2/10
7232/7232 [==============================] - 42s 6ms/step - loss: 0.3980 - acc: 0.8225 - val_loss: 0.3531 - val_acc: 0.8400
Epoch 3/10
7232/7232 [==============================] - 41s 6ms/step - loss: 0.3192 - acc: 0.8563 - val_loss: 0.2656 - val_acc: 0.8880
Epoch 4/10
7232/7232 [==============================] - 42s 6ms/step - loss: 0.2382 - acc: 0.9058 - val_loss: 0.2057 - val_acc: 0.9200
Epoch 5/10
7232/7232 [==============================] - 42s 6ms/step - loss: 0.1893 - acc: 0.9273 - val_loss: 0.1881 - val_acc: 0.9260
Epoch 6/10
1536/7232 [=====>........................] - ETA: 32s - loss: 0.1655 - acc: 0.9434